# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
weather_df = pd.read_csv("../WeatherPy/weather_data.csv")
weather_df.head()

,City,Lat,Lon,Max Temp (f),Humidity,Cloudiness,Wind Speed,Country,Date
0,Barrow,71.2906,-156.7887,46.42,76,75,12.66,US,1655440622
1,Kiryat Gat,31.6100,34.7642,77.81,100,0,7.92,IL,1655440848
2,Hobart,-42.8794,147.3294,52.27,87,99,1.99,AU,1655440642
3,Kumba,4.6363,9.4469,70.34,99,100,1.39,CM,1655440851
4,Bredasdorp,-34.5322,20.0403,50.23,45,64,6.08,ZA,1655440851


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
gmaps.configure(api_key=g_key)

locations = weather_df[["Lat", "Lon"]]
humidity = weather_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=100)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
weather_df2 = weather_df[(weather_df["Max Temp (f)"] >45) & (weather_df["Max Temp (f)"] <65) & (weather_df["Wind Speed"] <10) & (weather_df["Cloudiness"] ==0)]
weather_df2 = weather_df2.dropna().reset_index()
weather_df2

,index,City,Lat,Lon,Max Temp (f),Humidity,Cloudiness,Wind Speed,Country,Date
0,8,Vila Velha,-20.3297,-40.2925,63.95,90,0,0.00,BR,1655440706
1,69,Umeå,63.8284,20.2597,55.90,71,0,2.30,SE,1655440905
2,104,Arrondissement du Havre,49.5833,0.3333,64.80,72,0,8.05,FR,1655440937
3,111,Port Elizabeth,-33.9180,25.5701,48.49,71,0,2.30,ZA,1655440943
4,161,Torbay,47.6666,-52.7314,47.17,85,0,5.75,CA,1655440988
5,194,Coquimbo,-29.9533,-71.3436,49.73,93,0,6.91,CL,1655441016
6,247,Fort Frances,48.5999,-93.4003,61.02,68,0,4.61,CA,1655441061
7,278,Fortuna,40.5982,-124.1573,55.65,85,0,0.00,US,1655441089
8,354,Altamont,42.2068,-121.7372,57.04,51,0,6.91,US,1655441164
9,393,La Baule-Escoublac,47.2922,-2.3640,64.06,86,0,0.00,FR,1655441198


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
hotel_df = pd.DataFrame(weather_df2[["Country","City","Lat","Lon"]])

hotel_df["Hotel Name"] = ""

hotel_df

,Country,City,Lat,Lon,Hotel Name
0,BR,Vila Velha,-20.3297,-40.2925,
1,SE,Umeå,63.8284,20.2597,
2,FR,Arrondissement du Havre,49.5833,0.3333,
3,ZA,Port Elizabeth,-33.9180,25.5701,
4,CA,Torbay,47.6666,-52.7314,
5,CL,Coquimbo,-29.9533,-71.3436,
6,CA,Fort Frances,48.5999,-93.4003,
7,US,Fortuna,40.5982,-124.1573,
8,US,Altamont,42.2068,-121.7372,
9,FR,La Baule-Escoublac,47.2922,-2.3640,


In [25]:
skipped_city = []

for index, row in hotel_df.iterrows():
    try:
        coord = f"{row.Lat},{row.Lon}"
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        params = {"location": coord, "type": "lodging", "radius": 5000, "key": g_key}
        print(f"Retrieving Results for Index {index}: {coord}.")
        response = requests.get(base_url,params=params).json()
        results = response["results"]
        print(results[0]["name"])
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except:
        print("no hotel found within radius 5000 m")
        skipped_city.append(index)


Retrieving Results for Index 0: -20.3297,-40.2925.
Hotel Vitória Palace
Retrieving Results for Index 1: 63.8284,20.2597.
Hotel Avenue | ProfilHotels
Retrieving Results for Index 2: 49.5833,0.3333.
gite bois jourdain
Retrieving Results for Index 3: -33.918,25.5701.
39 On Nile Guest House
Retrieving Results for Index 4: 47.6666,-52.7314.
Gallows Cove
Retrieving Results for Index 5: -29.9533,-71.3436.
Hotel San Juan
Retrieving Results for Index 6: 48.5999,-93.4003.
LA PLACE RENDEZ-VOUS
Retrieving Results for Index 7: 40.5982,-124.1573.
Super 8 by Wyndham Fortuna
Retrieving Results for Index 8: 42.2068,-121.7372.
Microtel Inn & Suites by Wyndham Klamath Falls
Retrieving Results for Index 9: 47.2922,-2.364.
Hôtel Barrière Le Royal La Baule
Retrieving Results for Index 10: 47.4736,-94.8803.
Hampton Inn & Suites Bemidji
Retrieving Results for Index 11: -34.7333,135.8667.
Port Lincoln Hotel
Retrieving Results for Index 12: -17.3667,27.15.
no hotel found within radius 5000 m


In [26]:
for i in skipped_city:
    hotel_df = hotel_df.drop(i)

hotel_df

,Country,City,Lat,Lon,Hotel Name
0,BR,Vila Velha,-20.3297,-40.2925,Hotel Vitória Palace
1,SE,Umeå,63.8284,20.2597,Hotel Avenue | ProfilHotels
2,FR,Arrondissement du Havre,49.5833,0.3333,gite bois jourdain
3,ZA,Port Elizabeth,-33.9180,25.5701,39 On Nile Guest House
4,CA,Torbay,47.6666,-52.7314,Gallows Cove
5,CL,Coquimbo,-29.9533,-71.3436,Hotel San Juan
6,CA,Fort Frances,48.5999,-93.4003,LA PLACE RENDEZ-VOUS
7,US,Fortuna,40.5982,-124.1573,Super 8 by Wyndham Fortuna
8,US,Altamont,42.2068,-121.7372,Microtel Inn & Suites by Wyndham Klamath Falls
9,FR,La Baule-Escoublac,47.2922,-2.3640,Hôtel Barrière Le Royal La Baule


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]



In [28]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))